In [40]:
#Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os

In [2]:
#Path to chromedriver
!which chromedriver

/c/bin/chromedriver


In [3]:
# Set executable path and initialize the chrome browser in splinter
# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
executable_path = {'executable_path': 'C:/bin/chromedriver.exe'}

In [4]:
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [5]:
# Visit the mars nasa site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

#optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide" , wait_time=1)

True

In [6]:
#Convert the browser html to as soup object and then quit the browser
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')

slide_elem = news_soup.select_one('ul.item_list li.slide')

In [ ]:
print (news_soup.prettify())

In [7]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8622/virginia-middle-school-student-earns-honor-of-naming-nasas-next-mars-rover/" target="_self">Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover</a></div>

In [8]:
# use the parent element to find the first a tage and save it as 'news_title'
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover"

In [9]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find("div", class_='article_teaser_body').get_text()
news_p

'NASA chose a seventh-grader from Virginia as winner of the agency\'s "Name the Rover" essay contest. Alexander Mather\'s entry for "Perseverance" was voted tops among 28,000 entries. '

### JPL Mars Space Image

In [10]:
# Visit the NASA JPL (Jet Propulsion Laboratory) Site
executable_path = {'executable_path': 'C:/bin/chromedriver.exe'}
browser = Browser("chrome", **executable_path)


In [11]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [12]:
# Ask Splinter to Go to Site and Click Button with Class Name full_image
# <button class="full_image">Full Image</button>
full_image_button = browser.find_by_id("full_image")
full_image_button.click()

In [13]:
# Find "More Info" Button and Click It
browser.is_element_present_by_text("more info", wait_time=1)
more_info_element = browser.find_link_by_partial_text("more info")
more_info_element.click()

C:\Users\srue2\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [14]:
# Parse Results HTML with BeautifulSoup
html = browser.html
image_soup = BeautifulSoup(html, "html.parser")

In [17]:
featured_img_url = image_soup.select_one("figure.lede a img").get("src")
featured_img_url

'/spaceimages/images/largesize/PIA19639_hires.jpg'

In [19]:

# Use Base URL to Create Absolute URL
featured_img_url = f"https://www.jpl.nasa.gov{featured_img_url}"
print(featured_img_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19639_hires.jpg


### Mars Weather

In [20]:
# Visit the Mars Weather Twitter Account
executable_path = {'executable_path': 'C:/bin/chromedriver.exe'}
browser = Browser("chrome", **executable_path, headless=False)


In [21]:
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

In [22]:
# Parse Results HTML with BeautifulSoup
html = browser.html
weather_soup = BeautifulSoup(html, "html.parser")


In [23]:
# Scrape Mars Weather Twitter account for latest weather report on Mars
twitter_url = 'https://twitter.com/marswxreport?lang=en'
twitter_response = requests.get(twitter_url)
twitter_soup = BeautifulSoup(twitter_response.text, 'lxml')
twitter_result = twitter_soup.find('div', class_='js-tweet-text-container')


In [24]:
print(f"mars_weather ={twitter_result.text.strip()}")

mars_weather =InSight sol 453 (2020-03-05) low -95.1ºC (-139.1ºF) high -10.8ºC (12.6ºF)
winds from the SSW at 6.0 m/s (13.3 mph) gusting to 21.4 m/s (47.9 mph)
pressure at 6.30 hPapic.twitter.com/8nEk9RxzXk


### Mars Facts

In [45]:
# Visit the Mars Facts Site Using Pandas to Read
mars_df = pd.read_html("https://space-facts.com/mars/")[0]
mars_df.columns=["Description","Value"]
mars_df.set_index("Description",inplace=True)
mars_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [46]:
# Export pandas df to html script
marsfacts = mars_df.to_html()
marsfacts.replace("\n", "")
mars_df.to_html('marsfacts.html')

### Mars Hemispheres

In [26]:
# Visit the USGS Astrogeology Science Center Site
executable_path = {'executable_path': 'C:/bin/chromedriver.exe'}
browser = Browser("chrome", **executable_path, headless=False)

In [27]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [28]:
hemisphere_image_urls = []

# Get a List of All the Hemispheres
links = browser.find_by_css("a.product-item h3")
for item in range(len(links)):
    hemisphere = {}
    
    # Find Element on Each Loop to Avoid a Stale Element Exception
    browser.find_by_css("a.product-item h3")[item].click()
    
    # Find Sample Image Anchor Tag & Extract <href>
    sample_element = browser.find_link_by_text("Sample").first
    hemisphere["img_url"] = sample_element["href"]
    
    # Get Hemisphere Title
    hemisphere["title"] = browser.find_by_css("h2.title").text
    
    # Append Hemisphere Object to List
    hemisphere_image_urls.append(hemisphere)
    
    # Navigate Backwards
    browser.back()

C:\Users\srue2\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:536: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  FutureWarning,


In [32]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]